# American options pricing

This approach uses the following general procedure:

1. Compute the **expected value** at each node and compare it with the **instrinsic value**. 

2. The maximum of both those values is the **final intermediate value** at a particular node. 

3. We then iterative backwards in time to get the value TODAY.

In [1]:
import numpy as np

In [2]:
def APO(ST, K):
    if ST < K:
        payoff = K - ST
    elif ST >= K:
        payoff = 0
    return payoff

In [4]:
def draw_tree(ts, S0):
    d1 = {}
    d2 = {}
    for t in range(ts):
        if t == 0:
            d1[t] = 1
        elif t > 0:
            d1[t] = d1[t-1]*2
        d2[t] = []

    d2[0].append(S0)
    
    return d1, d2

In [25]:
D1, D2 = draw_tree(5, 4)

In [6]:
D1

{0: 1, 1: 2, 2: 4, 3: 8}

In [7]:
def simulate_values(ts, D_1, D_2, u, d, K):
    for t in range(1, ts):
        for i in range(D_1[t-1]):
            D_2[t].append(D_2[t-1][i]*u)
            D_2[t].append(D_2[t-1][i]*d)
            
    D_2[len(D_2.keys())-1] = [APO(x, K) for x in D_2[len(D_2.keys())-1]]
    return D_2

In [26]:
dd2 = simulate_values(5, D1, D2, 2, 0.5, 5)

In [40]:
dd2

{0: [4], 1: [8, 2.0], 2: [0, 1.0, 1.0, 4.0]}

In [18]:
def backward_dynamic_AMERICAN(dd, q, r):
    h = {}
    for t in sorted(list(dd.keys()), reverse=True):
        if t == 0:
            break

        arr = []
        for i in range(len(dd[t])):
            if i % 2 == 0:
                exp = ((q*dd[t][i]) + ((1-q)*dd[t][i+1]))/(1.25)
                iv = APO(dd[t-1][i//2], 5)
                val = max(exp, iv)
                arr.append(round(val, 7))
            elif i % 2 != 0:
                pass

        h[t-1] = arr
        dd[t-1] = arr
        
    return h

In [13]:
((1/2)*(0.48))/(1.25) + ((1/2)*(3))/(1.25)

1.392

In [27]:
prices = backward_dynamic_AMERICAN(dd2, 0.5, 0.25)
#prices

In [28]:
# extract price

print("price of the APO is: ", prices[0])

price of the APO is:  [1.4432]
